# Liaison Data et Ontologie

Utilisation de ChatGPT pour extraire les thèmes puis instanciations des données dans l'ontologie via owlready2

## Imports

In [1]:
try:
  import openai
  import json
  from ast import literal_eval
  from tqdm import tqdm
  from unidecode import unidecode
  import owlready2 as owl
  import types
  import os
except ModuleNotFoundError:
  %pip install -r "requirements.txt"

in_ontology_url = "https://raw.githubusercontent.com/ApprocheSemantiqueP2/ontologie/main/Music_ID_ontology.owl"
out_ontology_url = "./ontologie_finale.owl"
openai.api_key = os.environ["OPENAI_API_KEY"]

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


## Extractions des thèmes

### Utilisation de la bibliothèque openai

In [3]:
class LyricsThemeExtractor:
    def __init__(self) -> None:
        self.system_prompt = """"
        Tu es un assistant textuel qui extrait les thèmes en un seul mot correspondants aux paroles d'une chanson mot par mot, les extrayant dans une liste python au format suivant: ["<theme1>", "<theme2>", ...].
        Retourne moi seulement la liste, aucun texte autour sinon c'est invalide.
        Extrait le plus de thèmes possible.
        Essaye de te restreindre aux thèmes suivants: ["Amitié", "Amour", "Amusement", "Animosité", "Argent", "Armes", "Drogue", "Energie", "Epreuve", "Fantasy", "Histoire", "Joie", "Peine", "Religion", "Trahison"] mais tu peux en rajouter si besoin.
        """

    def get_themes(self, paroles: str, max_retries=3):
        user_prompt = f'Extrait les themes EN FRANÇAIS SEULEMENT des paroles suivantes en UN SEUL MOT sinon invalide "{paroles}"'
        res = None
        retries = 0

        while res == None and retries < max_retries:
          try:
            completion = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": self.system_prompt},
                    {"role": "user", "content": user_prompt}
                ]
            )
            res = literal_eval(completion["choices"][0]["message"]["content"])
          except:
            retries += 1
        
        return res

extractor = LyricsThemeExtractor()

### Exemple

In [21]:
paroles = """
COUPLET

Mon reuf me dis que tout va bene
Mais fuck cette vie j’attends bénéfice
Un keuf qui tire y’a du délai
Une goutte pour les morts, vide le Hennessy
La guerre nous on sait que c’est khene
Y’a que pour la vente d’armes que c’est bénéfique
Les snakes autour de nous qui dansent
Toutes ces plaies qu’on panse
Nous rendent frénétique
Yeah
Un million de galères qu’on a mis sur le tec
Ma voix dans un Pultec
J’veux m’barrer j’ai déjà fait le tour de la zone
Les zombies qui traient on les connaît les shlags les gros bonnets ah yeah À la misère abonné ah yeah
J’attends juste de récupérer ma somme
REFRAIN

Ma gasolina part et dans ma tête je rembobine
J’viens pas de ce milieu, zéro pistons donc on improvise
J’ai fais le tour de la ville, les poches et le ventre vide
En catimini on s’prépare pour nous rien n’est impossible

COUPLET 2

Plus d’coco dans le coeur, c’est la grève
Ma gasolina veut tirer un trait
Un ordi, une carte et un micro
Y’a des secousses on vient poser l’adresse
Tous les jours 75 aller-retour paris sud, paris nord j’crame
Autour que des plavans chelous qui s’trament
J’les vois cons’ le que-cra j’les vois qu’ils se canent
Dans l’blizzard on navigue encore à l’aveugle
Tout pour soigner l’entorse yeah
J’dois réparer son cœur en porcelaine
Un peu d’carburant dans l’amphore

REFRAIN

Ma gasolina part et dans ma tête je rembobine
J’viens pas de ce milieu, zéro pistons donc on improvise
J’ai fais le tour de la ville, les poches et le ventre vide
En catimini on s’prépare pour nous rien n’est impossible
"""

res = extractor.get_themes(paroles)
print(res)

['Argent', 'Armes', 'Drogue']


### Extraction des thèmes

In [22]:
f = open("music_id.json", "r")
data = json.load(f)

for song in tqdm(data):
    themes = extractor.get_themes(song["lyrics"].replace("\n", " "))
    if themes is not None:
        song["theme"] = [unidecode(x).capitalize() for x in themes]

f2 = open("data_final.json", "w")
json_obj = json.dumps(data, indent=4)

f2.write(json_obj)

100%|██████████| 140/140 [04:35<00:00,  1.97s/it]


1826613

## Instanciation des données

In [2]:
f = open("data_final.json", "r")

onto = owl.get_ontology(in_ontology_url).load()
data = json.load(f)

for song in data:
  newMorceau = onto.music(song['track'].replace(" ", "_").upper(),
                          songname = [song["track"]],
                          Duration = [song["duration_ms"]],
                          Songpop = [song["popularity"]], 
                          BPM = [song["tempo"]])

  if song["album"] == song["track"]:
    newRelease = onto.Single(song['album'].replace(" ", "_").upper(), releasename = [song["album"]], Date_de_sortie = [song["release_date"]])
  else:
    newRelease = onto.Album(song['album'].replace(" ", "_").upper(), releasename = [song["album"]], Date_de_sortie = [song["release_date"]])
  newRelease.Contient.append(newMorceau)

  newAuteur = onto.Auteur(song['artist'][0].replace(" ", "_").upper(), Nom = [song["artist"][0]])
  
  for x in [newMorceau, newRelease]:
    x.aPourCreateur.append(newAuteur)
  
  for artist in song["artist"] + song["producer_artist"] + song["writer_artist"]:
    newArtiste = onto.Artiste(artist.replace(" ", "_").upper(), Nom = [artist])
    newArtiste.aSorti.append(newRelease)
  if song["theme"] is not None:
    for theme in song["theme"]:
      with onto:
        try:
          newThemeClass = types.new_class(theme.replace(" ", "_"), (onto["Thème"], ))
          newMorceau.aPourTheme.append(newThemeClass)
        except TypeError:
          continue

    newParoles = onto.Paroles(song["track"].replace(" ", "_").upper(), Contenu = [song["lyrics"]], Langue = [song["language"]])

onto.save(file = out_ontology_url, format = "rdfxml")